In [2]:
%load_ext autotime

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["SUFFIX"]= "1500K-with-3-vids"
os.environ["BASE_DIR"] = "/home/ec2-user/emb3"

hyperparameters = {
    'loss': "adaptive_hinge",
    'batch_size': 256,
    'learning_rate': 1e-3,
    'l2': 1e-06,
    'n_iter': 30,
    'embedding_dim': 64
}

import pandas as pd
import numpy as np
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.evaluation import mrr_score
from spotlight.factorization.implicit import ImplicitFactorizationModel
from spotlight.interactions import Interactions
import pandas as pd
from spotlight.evaluation import *
from torch.utils.tensorboard import SummaryWriter
import torch
from torch.utils.tensorboard import SummaryWriter
import logging
from spotlight.factorization.representations import *
import os
import collections

from spotlight.sequence.implicit import ImplicitSequenceModel
from spotlight.sequence.representations import CNNNet
from spotlight.torch_utils import *

In [3]:
mm = torch.load("/home/ec2-user/emb3/smodels/cnn_loss=adaptive_hinge,batch=128,lr=0.001,l2=1e-06,n_iter=50,emb_dim=128,type=cnn,mom=0.9,neg=3,amsgrad=True,adamw=False-0")

time: 1.7 s


In [4]:
mm

CNNNet(
  (item_embeddings): ScaledEmbedding(17540, 128, padding_idx=0)
  (item_biases): ZeroEmbedding(17540, 1, padding_idx=0)
  (cnn_0): Conv2d(128, 128, kernel_size=(3, 1), stride=(1, 1))
)

time: 7.51 ms


In [5]:
suffix = "1500K-with-3-vids"

time: 408 µs


In [6]:
train_data_path = "/home/ec2-user/emb3/data/train-aug-28-phase" + suffix
original_train_data = pd.read_parquet(train_data_path)
validate_neg_flatten_vids = pd.read_parquet(os.environ['BASE_DIR'] + "/data/validate-neg-flatten-aug-28-phase" + suffix)
validate_pos_flatten_vids = pd.read_parquet(os.environ['BASE_DIR'] + "/data/validate-pos-flatten-aug-28-phase" + suffix)
validation_train_data = original_train_data[original_train_data["uindex"].isin(validate_pos_flatten_vids.uindex.tolist())]

time: 9.15 s


In [7]:
validation_train_data["vindex"] = validation_train_data["vindex"] + 1

# sort_indices = np.lexsort((validation_train_data["latest_watch_time"].tolist(), validation_train_data["uindex"].tolist()))
# vlist = validation_train_data.iloc[sort_indices].groupby("uindex")["vindex"].agg(list).reset_index()
# vlist.to_parquet(os.environ['BASE_DIR'] + "/data/" + "loo-validation-" + suffix)

time: 4.29 ms


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [8]:
interactions = Interactions(validation_train_data["uindex"].to_numpy(),
            validation_train_data["vindex"].to_numpy(),
            validation_train_data["pct_cvt"].to_numpy(),
            validation_train_data["latest_watch_time"].to_numpy(),
            num_users=len(original_train_data["uindex"].unique()),
            num_items=len(original_train_data["vindex"].unique()) + 2)

max_sequence_len=100
sequences = interactions.to_sequence(max_sequence_length=max_sequence_len, step_size=max_sequence_len)

time: 1.42 s


In [10]:
#sequences.sequences
use_cuda= True
sqs = sequences.sequences[:5]
sequences_tensor = gpu(torch.from_numpy(sqs).long(),use_cuda)
user_representation, _ = mm.user_representation(sequences_tensor)

vids=[1,3, 4,5,7]

vids_tensor = gpu(torch.from_numpy(np.array(vids).reshape(-1, 1)).long(),use_cuda)


time: 2.15 ms


time: 1.14 ms


In [16]:
mm(user_rp[1], vids_tensor)

tensor([ 3.5602, -0.0070,  1.3687,  1.3683,  2.3950], device='cuda:0',
       grad_fn=<AddBackward0>)

time: 6.65 ms


In [33]:
layers = [256, 128, 64, 1]

fc_layers =  None
if layers and len(layers) > 0:
    fc_layers = torch.nn.ModuleList()
    for idx, (in_size, out_size) in enumerate(zip(layers[:-1], layers[1:])):
        fc_layers.append(torch.nn.Linear(in_size, out_size))

time: 2.57 ms


In [41]:
fc_layers = gpu(fc_layers, True)

time: 1.5 ms


In [24]:
fc_layers

ModuleList(
  (0): Linear(in_features=128, out_features=64, bias=True)
  (1): Linear(in_features=64, out_features=32, bias=True)
  (2): Linear(in_features=32, out_features=1, bias=True)
)

time: 2.35 ms


In [52]:
vids=[1,3, 4,5,7]

user_representations = user_rp
target_embedding = mm.item_embeddings(gpu(torch.from_numpy(np.array(vids).reshape(-1, 1)).long(),use_cuda)).permute(0, 2, 1).squeeze()

vector = torch.cat([user_representations, target_embedding], dim=-1)

nonlinearity = torch.nn.functional.relu
if fc_layers:
    for idx, _ in enumerate(range(len(fc_layers)-1)):
        vector = nonlinearity(fc_layers[idx](vector))
    final_output = fc_layers[-1](vector)

time: 9.08 ms


In [57]:
final_output.squeeze().shape

torch.Size([5])

time: 2.24 ms


In [55]:
dot = ((user_representations * target_embedding)
               .sum(1))
dot.shape

torch.Size([5])

time: 8.93 ms


In [59]:
[int(x) for x in "256-128-64-1".split("-")]

[256, 128, 64, 1]

time: 2.92 ms


In [45]:
for idx, _ in enumerate(range(len(fc_layers)-1)):
    print(idx)

0
1
2
time: 848 µs


In [145]:
iit = list(zip(sequences.user_ids, sequences.sequences))


time: 40.7 ms


In [ ]:
validate_neg_flatten_vids = pd.read_parquet(os.environ['BASE_DIR'] + "/data/validate-neg-flatten-aug-28-phase" + suffix)
validate_pos_flatten_vids = pd.read_parquet(os.environ['BASE_DIR'] + "/data/validate-pos-flatten-aug-28-phase" + suffix)

validate_neg_flatten_vids["vindex"] = validate_neg_flatten_vids["vindex"]+1
validate_pos_flatten_vids["vindex"] = validate_pos_flatten_vids["vindex"]+1

merged_valid_vids = validate_neg_flatten_vids.groupby("uindex")["nvindex"].agg(list).reset_index().merge(validate_pos_flatten_vids, on="uindex")
merged_valid_vids["videos"] = merged_valid_vids.apply(lambda x: x["nvindex"] + [x["vindex"]], axis=1)
merged_valid_vids = merged_valid_vids.set_index("uindex")
merged_valid_vids["videos"]


In [186]:
import gc

time: 482 µs


In [2]:
sequences_tensor

NameError: name 'sequences_tensor' is not defined

time: 164 ms


In [231]:
use_cuda=True
iit = list(zip(sequences.user_ids, sequences.sequences))

bs = 1024

pos_uids = []
pos_item_ids = []
pos_scores = []
neg_uids = []
neg_item_ids = []
neg_scores = []

cnt=0

for uid, seqs in iit:
    cnt+=1
    print(cnt)
    sequences_tensor = gpu(torch.from_numpy(sqs).long(),use_cuda)
    user_rp = mm.user_representation(sequences_tensor)[1]
    

# for i in list(range(0, len(iit), bs)):
#     lb, ub = i, i+bs
#     uids, seqs = zip(*iit[lb:ub])
#     sequences_tensor = gpu(torch.from_numpy(sqs).long(),use_cuda)
#     user_rp = mm.user_representation(sequences_tensor)[1]
    
#     for i in range(len(uids)):
#         uid = uids[i]
#         vids = merged_valid_vids["videos"][uid]
#         predictions = mm(fuser_rp[i].expand((len(vids), 128)), gpu(torch.from_numpy(np.array(vids).reshape(-1, 1)).long(),use_cuda)).cpu().detach().numpy()
    
#     user_rp= None
#     sequences_tensor= None
#     gc.collect()
#     torch.cuda.empty_cache()

1


RuntimeError: CUDA out of memory. Tried to allocate 400.00 MiB (GPU 0; 11.17 GiB total capacity; 9.62 GiB already allocated; 259.81 MiB free; 1.00 GiB cached)

time: 99.8 ms


In [226]:
user_rp[0]

torch.Size([128])

time: 2.27 ms


In [198]:
final_user_rep[35].expand((25, 128))

torch.Size([25, 128])

time: 2.54 ms


In [209]:
#neg_vids = validate_neg_flatten_vids.merge(validate_pos_flatten_vids, on="uindex").groupby("uindex")["nvindex"].agg(list)


uindex
35         [4386, 8400, 10425, 8402, 2178, 17263, 8761, 1...
67         [15940, 1747, 8325, 14425, 7237, 11847, 17162,...
113        [5857, 12046, 6426, 8102, 1471, 5340, 5353, 17...
125        [8905, 6958, 5109, 118, 17193, 12712, 9036, 16...
150        [16104, 11892, 9863, 2721, 11494, 14878, 15662...
                                 ...                        
1498702    [15894, 9131, 1424, 5444, 7875, 12222, 11277, ...
1498707    [3580, 15871, 451, 12339, 13824, 12737, 997, 1...
1498711    [3170, 5622, 6511, 15128, 6487, 8693, 10238, 1...
1498734    [6808, 15991, 8031, 14241, 14683, 17451, 15162...
1498783    [3352, 17060, 8419, 10223, 14741, 11400, 16829...
Name: videos, Length: 75000, dtype: object

time: 11.7 s


time: 2.01 ms


In [179]:
len(iit)

76091

time: 2 ms


In [178]:
lb
ub
#uids, seqs = zip(*iit[lb:ub])

67117056

time: 2.08 ms


In [172]:
iit = list(zip(sequences.user_ids, sequences.sequences))

sqs = sequences.sequences[0:8192]
use_cuda=True
sequences_tensor = gpu(torch.from_numpy(sqs).long(),use_cuda)

mm.user_representation(sequences_tensor)[1].shape

torch.Size([8192, 128])

time: 63.1 ms


In [168]:
sequences_tensor.shape

torch.Size([8192, 100])

time: 2.11 ms


In [173]:
dd = dict(zip(range(len(sequences_tensor)), mm.user_representation(sequences_tensor)[1]))
torch.stack([dd[0], dd[1]]).shape

torch.Size([2, 128])

time: 37.8 ms


In [139]:
validate_neg_flatten_vids = pd.read_parquet(os.environ['BASE_DIR'] + "/data/validate-neg-flatten-aug-28-phase" + suffix)
validate_pos_flatten_vids = pd.read_parquet(os.environ['BASE_DIR'] + "/data/validate-pos-flatten-aug-28-phase" + suffix)

time: 187 ms


,uindex,nvindex,vindex
0,389020,2681,13849
1,389020,2724,13849
2,389020,9647,13849
3,389020,12293,13849
4,389020,13562,13849
5,389020,15320,13849
6,389020,16516,13849
7,389020,8739,13849
8,389020,6449,13849
9,389020,11069,13849


time: 691 ms


In [112]:
gpu(torch.from_numpy(vlist["vindex"][0:2].to_numpy()), True)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, int64, int32, int16, int8, uint8, and bool.

time: 10.6 ms


In [125]:
#np.matrix(vlist["vindex"][0:2].tolist())
np.stack(vlist["vindex"][0:2].values)

ValueError: all input arrays must have the same shape

time: 11.5 ms


In [105]:
sequences_tensor = gpu(torch.from_numpy(sqs).long(),use_cuda)

75000

time: 2.29 ms


In [23]:
train_data_path = "/home/ec2-user/emb3/data/train-aug-28-phase" + suffix
original_train_data = pd.read_parquet(train_data_path)
uvs = original_train_data.groupby("uindex")["vindex"].agg(list)
max_sequence_len = 50
#uvs.loc[validate_pos_flatten_vids.uindex.tolist()].apply(lambda x: len(x)).max()

train_data = original_train_data[original_train_data.uindex.isin(uvs[uvs.apply(lambda x: len(x)) <= max_sequence_len].index)]
train_data["vindex"] = train_data["vindex"]+1

time: 3min 7s


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [25]:
validate_neg_flatten_vids = pd.read_parquet(os.environ['BASE_DIR'] + "/data/validate-neg-flatten-aug-28-phase" + suffix)
validate_pos_flatten_vids = pd.read_parquet(os.environ['BASE_DIR'] + "/data/validate-pos-flatten-aug-28-phase" + suffix)

time: 226 ms


In [31]:
validation_train_data = train_data[train_data["uindex"].isin(validate_pos_flatten_vids.uindex.tolist())]

time: 6 s


In [41]:
interactions = Interactions(validation_train_data["uindex"].to_numpy(),
            validation_train_data["vindex"].to_numpy(),
            validation_train_data["pct_cvt"].to_numpy(),
            validation_train_data["latest_watch_time"].to_numpy(),
            num_users=len(original_train_data["uindex"].unique()),
            num_items=len(original_train_data["vindex"].unique()) + 2)

sequences = interactions.to_sequence(max_sequence_length=max_sequence_len, step_size=max_sequence_len)

time: 927 ms


In [70]:
from spotlight.torch_utils import cpu, gpu, minibatch, set_seed, shuffle

time: 580 µs


In [81]:
sqs = sequences.sequences[0:2]
use_cuda=True
sequences_tensor = gpu(torch.from_numpy(sqs).long(),use_cuda)

mm.user_representation(sequences_tensor)[0].shape

torch.Size([2, 128, 50])

time: 3.54 ms


In [42]:
max_sequence_length=50
sequences = interactions.to_sequence(max_sequence_length=50, step_size=50)

time: 506 ms


In [49]:
len(set(sequences.user_ids).intersection(validation_train_data.uindex.unique().tolist())), len(validation_train_data.uindex.unique())

(70982, 70982)

time: 182 ms


In [51]:
sequences.sequences[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,  2037, 17460, 12020,  5345, 16440,  3443,
        5250, 13248, 11933,  4393,  1950,  5314, 17458,  7239,  7669,
         363, 11228, 14032,  9753, 10334], dtype=int32)

time: 2.52 ms


In [13]:
def pairs_ndcg_and_recall_score(embs):
    suffix = os.environ['SUFFIX']
    vindex_pairs_df = pd.read_parquet(os.environ['BASE_DIR'] + "/data/test-pairs-indexed-aug-28-phase" + suffix)
    embs_ranks = calc_embs_rank(embs)
    number_of_videos = len(embs_ranks)
    lookup_table = embs_ranks.values.ravel()
    
    rrank = vindex_pairs_df.apply(lambda r: lookup_table[r["v1"] * number_of_videos + r["v2"]], axis=1)
    vindex_pairs_df["rrank"] = rrank
    
    ndcg_vals = vindex_pairs_df.apply(
        lambda r: (1. / np.log(r["rrank"] + 1.)) * r["count"], axis=1)
    recall_vals = vindex_pairs_df.apply(lambda r: (1 if r["rrank"] <= 10 else 0) * r["count"] ,axis=1)
    return ndcg_vals.mean(), recall_vals.mean()

time: 1.99 ms


In [14]:
suffix = os.environ['SUFFIX']
als_embds = pd.read_parquet("/home/ec2-user/emb3/data/als-embs-pandas-aug-28-phase" + suffix)
aa = pd.read_parquet(os.environ['BASE_DIR'] + "/data/video2index-pandas-aug-28-phase" + suffix)
videoid2index = dict(zip(aa["k"], aa["v"]))

number_of_videos = len(videoid2index)
embs = np.ma.masked_all((number_of_videos, 100))
for i, row in als_embds.iterrows():
    vindex = row["vindex"]
    if vindex != -1:
        embs[vindex, :] = row["vector"]
pairs_ndcg_and_recall_score(embs)

/home/ec2-user/emb3/spotlight/spotlight/evaluation.py:71: RuntimeWarning: divide by zero encountered in true_divide
  embeds_norm = np.divide(embeds, np.sqrt(np.square(embeds).sum(axis=1)).reshape(-1, 1))
/home/ec2-user/emb3/spotlight/spotlight/evaluation.py:71: RuntimeWarning: invalid value encountered in true_divide
  embeds_norm = np.divide(embeds, np.sqrt(np.square(embeds).sum(axis=1)).reshape(-1, 1))


(0.28422729905963035, 0.05332454594795479)

time: 2min 43s


In [16]:
model = mm
suffix = os.environ['SUFFIX']
aa = pd.read_parquet(os.environ['BASE_DIR'] + "/data/video2index-pandas-aug-28-phase" + suffix)
videoid2index = dict(zip(aa["k"], aa["v"]))

model.eval()
number_of_videos = len(videoid2index)
with torch.no_grad():
    raw_embeds = model.get_embeddings().detach()
    raw_embeds = raw_embeds.cpu()
    raw_embeds = raw_embeds.numpy()
    embed_size = model.get_embedding_size()

    embs = np.ma.masked_all((number_of_videos+10, embed_size))
    for idx, emb in enumerate(raw_embeds):
        #if idx in valid_ids:
        embs[idx, :] = emb
pairs_ndcg_and_recall_score(embs)

(0.2936994845841601, 0.06328120613548437)

time: 2min 41s


In [6]:
%ls /home/ec2-user/models/cnn_loss*

ls: cannot access /home/ec2-user/models/cnn_loss*: No such file or directory
time: 116 ms


In [4]:
print(torch.cuda.device_count())

1
time: 648 µs


In [5]:
suffix = "500K-with-3-vids"
train_data_path = "/home/ec2-user/emb3/data/train-aug-28-phase" + suffix
original_train_data = pd.read_parquet(train_data_path)

#train_data = original_train_data
train_data = original_train_data.copy()
train_data["vindex"] = train_data["vindex"]+1

time: 328 ms


In [6]:
interactions = Interactions(train_data["uindex"].to_numpy(),
            train_data["vindex"].to_numpy(),
            train_data["pct_cvt"].to_numpy(),
            train_data["latest_watch_time"].to_numpy(),
            num_users=len(original_train_data["uindex"].unique()),
            num_items=len(original_train_data["vindex"].unique()) + 1)

time: 348 ms


In [7]:
max_sequence_length=100
min_sequence_length=1
step_size=1
train_seq = interactions.to_sequence(max_sequence_length=max_sequence_length,
                              min_sequence_length=min_sequence_length,
                              step_size=step_size)

time: 22.1 s


In [ ]:
model_alias = ",".join([k+"="+str(v) for k,v in collections.OrderedDict(hyperparameters).items()])
model_alias = model_alias + "__TMP"
tensorboard_base_dir="/home/ec2-user/emb3/ttest"
writer = SummaryWriter(log_dir='{}/{}'.format(tensorboard_base_dir, model_alias))

model_store_dir= "/home/ec2-user/emb3/models"

def notify_loss_completion(epoch_id, batch_id, loss, net, model):
    #print("notify_loss_completion")
    writer.add_scalar("Batch/loss", loss, batch_id)
    print('[Epoch {}] Batch {}, Loss {}'.format(epoch_id, batch_id, loss))
    logging.info('[Epoch {}] Batch {}, Loss {}'.format(epoch_id, batch_id, loss))

def notify_batch_eval_completion(epoch_id, batch_id, loss, net, model):
    #print("notify_batch_eval_completion")
#     pairs_ndcg = nn_pairs_ndcg_score(net)
#     writer.add_scalar("Batch/pairs_ndcg", pairs_ndcg, batch_id)
#     logging.info('[Epoch {}] Batch {}, Embs NDCG = {:.4f}'.format(epoch_id, batch_id, pairs_ndcg))
#     print('[Epoch {}] Batch {}, Embs NDCG = {:.4f}'.format(epoch_id, batch_id, pairs_ndcg))
    print("Here")
    net.train()
    
def notify_epoch_completion(epoch_num, total_loss, net, model):
    #print("notify_epoch_completion")
    writer.add_scalar("Epoch/loss", total_loss, epoch_num)
    pairs_ndcg = nn_pairs_ndcg_score(net)
    writer.add_scalar("Epoch/pairs_ndcg", pairs_ndcg, epoch_num)
#     hit_ratio, ndcg = evaluate_hit_ratio_and_ndcg(model)
#     writer.add_scalar("Epoch/HR", hit_ratio, epoch_num)
#     writer.add_scalar("Epoch/NDCG", ndcg, epoch_num)
    hit_ratio, ndcg = -1,-1
    logging.info('******** [Epoch {}]  Embs NDCG {:.4f}, Hit Ratio: {:.4f}, NDCG: {:.4f}'.format(epoch_num,
                                                                                                    pairs_ndcg,
                                                                                                    hit_ratio,
                                                                                                    ndcg))
    print('******** [Epoch {}]  Embs NDCG {:.4f}, Hit Ratio: {:.4f}, NDCG: {:.4f}'.format(epoch_num,
                                                                                                    pairs_ndcg,
                                                                                                    hit_ratio,
                                                                                                    ndcg))
    torch.save(net, model_store_dir + "/" + model_alias + "-" + str(epoch_num))
    net.train()


os.environ["SUFFIX"] = suffix
os.environ["BASE_DIR"] = "/home/ec2-user/emb3"

writer.add_text('alias', model_alias, 0)

CUDA=True
h = hyperparameters
random_state = np.random.RandomState(100)

model = ImplicitSequenceModel(loss=h['loss'],
                              representation='lstm',
                              batch_size=h['batch_size'],
                              learning_rate=h['learning_rate'],
                              l2=h['l2'],
                              n_iter=h['n_iter'],
                              embedding_dim=h['embedding_dim'],
                              use_cuda=CUDA,
                              random_state=random_state,
                             notify_loss_completion=notify_loss_completion,
                                  notify_batch_eval_completion=notify_batch_eval_completion,
                                  notify_epoch_completion=notify_epoch_completion,
                             log_loss_interval=5000,
                             log_eval_interval=20000)

model.fit(train_seq, verbose=True)

[Epoch 0] Batch 0, Loss 1.008164405822754
Here
[Epoch 0] Batch 5000, Loss 0.4232798805832863
[Epoch 0] Batch 10000, Loss 0.2841775295913219


In [7]:
m = torch.load("models/64-4-bpr-MLP-suf-500K-with-3-vids-loss-adaptive_bpr-lr-0.001-l2-1e-05-mom-0.99-smpl-1.0-ng-3-bt-1024-drp-None-amsgrad-True-adamw=None-0")

time: 1.88 s


In [15]:
m.embedding_item.weight[1:,:].shape, m.embedding_item.weight.shape

(torch.Size([17388, 128]), torch.Size([17389, 128]))

time: 2.62 ms


In [6]:
suffix = "500K-with-3-vids"
os.environ["SUFFIX"] = suffix
os.environ["BASE_DIR"] = "/home/ec2-user/emb3"

time: 699 µs


In [8]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.evaluation import mrr_score
from spotlight.factorization.implicit import ImplicitFactorizationModel

dataset = get_movielens_dataset(variant='1M')

train, test = random_train_test_split(dataset)


time: 150 ms


In [9]:
model = ImplicitFactorizationModel(n_iter=15,
                                   loss='bpr')
model.fit(train)

mrr = mrr_score(model, test)

time: 6min 8s


In [10]:
mrr.mean()

0.02882165588353406

time: 2.52 ms


In [ ]:
NUM_SAMPLES = 100

LEARNING_RATES = [1e-3, 1e-2, 5 * 1e-2, 1e-1]
LOSSES = ['bpr', 'hinge', 'adaptive_hinge', 'pointwise']
BATCH_SIZE = [8, 16, 32, 256]
EMBEDDING_DIM = [8, 16, 32, 64, 128, 256]
N_ITER = list(range(5, 20))
L2 = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 0.0]


def sample_cnn_hyperparameters(random_state, num):

    space = {
        'n_iter': N_ITER,
        'batch_size': BATCH_SIZE,
        'l2': L2,
        'learning_rate': LEARNING_RATES,
        'loss': LOSSES,
        'embedding_dim': EMBEDDING_DIM,
        'kernel_width': [3, 5, 7],
        'num_layers': list(range(1, 10)),
        'dilation_multiplier': [1, 2],
        'nonlinearity': ['tanh', 'relu'],
        'residual': [True, False]
    }

    sampler = ParameterSampler(space,
                               n_iter=num,
                               random_state=random_state)

    for params in sampler:
        params['dilation'] = list(params['dilation_multiplier'] ** (i % 8)
                                  for i in range(params['num_layers']))

        yield params


def sample_lstm_hyperparameters(random_state, num):

    space = {
        'n_iter': N_ITER,
        'batch_size': BATCH_SIZE,
        'l2': L2,
        'learning_rate': LEARNING_RATES,
        'loss': LOSSES,
        'embedding_dim': EMBEDDING_DIM,
    }

    sampler = ParameterSampler(space,
                               n_iter=num,
                               random_state=random_state)

    for params in sampler:

        yield params

In [14]:
max_sequence_length=200
min_sequence_length=2
step_size=2

train_seq = train.to_sequence(max_sequence_length=max_sequence_length,
                              min_sequence_length=min_sequence_length,
                              step_size=step_size)
test_seq = test.to_sequence(max_sequence_length=max_sequence_length,
                        min_sequence_length=min_sequence_length,
                        step_size=step_size)

time: 1.6 s
